## neo4j 图数据库操作

### 导入相关依赖

```bash
uv add py2neo # neo4j 的 python 客户端
uv add tqdm # 进度条显示库
uv add prettyprinter # 增强的数据格式化输出库
```

In [1]:
from py2neo import Graph, Node, Relationship

In [2]:
import json
from tqdm import tqdm
from pprint import pprint

### 连接到 Neo4j 数据库

In [5]:
graph = Graph("bolt://localhost:7687", user="neo4j", password="neo4j123", name="neo4j")

### 解析数据

#### 公司事件数据解析

In [16]:
event_type_file = './data/iree.json'
event_type_data =[]
with open(event_type_file, 'r', encoding='utf-8') as file:
    for line in file.readlines():
        event_type_data.append(json.loads(line.strip()))

pprint(event_type_data[2], sort_dicts=True)

{'content': '据美国《纽约时报》29日报道，全球航空业放缓正对波音公司造成巨大打击。波音公司在29日表示，将裁员约1.6万人，此前该公司报告称，今年前三个月收入锐减26%。波音首席执行官戴维·卡尔霍恩在给员工的一份报告中说，“全球疫情大流行改变了人们的生活和工作方式。它正在改变我们的行业，我们面临着完全意想不到的挑战。”一家行业商会称，全世界的航空公司都在努力生存，到今年年底，航空业亏损总额预计将超过3000亿美元。因此，许多航空公司正在推迟购买、交付和维修。波音公司表示，正在放缓飞机的生产，包括陷入困境的737 '
            'Max、787、777和777x机型。该公司还在探索从美国联邦政府或金融市场筹集更多资金的方法。此次裁员约占波音员工总数的10%，对于那些受经济衰退影响最大的部门——商用飞机和服务行业——的员工来说，裁员幅度将更大，这些部门的裁员幅度约为15%。卡尔霍恩在报告中对雇员表示：“我知道，在一个已经充满挑战的时期，这个消息是一个打击。我很遗憾这会对你们当中的许多人产生影响。我衷心希望还有其他方法。”波音公司报告第一季度净亏损6.41亿美元，而去年同期则盈利21亿美元。该公司此前曾表示，预计航空旅行在三年内不会恢复到疫情暴发前的水平，而且很可能还需要几年时间，旅行才能恢复到以前的长期增长率。波音的商用飞机业务在第一季度受到737 '
            'Max机型和新冠病毒大流行的打击尤其严重，该业务的收入比去年同期下降近50%，降至62亿美元。公司总收入下降至169亿美元。该公司在1月至3月仅收到49笔新订单，有196笔订单被取消。西南航空(Southwest '
            'Airlines)在28日表示，该公司一直在就减少今年将接收的737 '
            'Max飞机数量进行谈判。西南航空表示，到2021年底，该公司最多将接收48架Max喷气式飞机，而不是此前预期的107架。波音表示，希望通过包括买断和提前退休的自愿方式来实现裁员目标。波音公司上周告诉员工，接受买断的员工依照在波音工作时长，每年将获得3个月的医疗保险和1周的工资，最高可达26年。员工必须在周一之前表明他们对买断的兴趣。如果获得批准，他们将在6月初离职。报道称，波音的裁员将可能不成比例地集中在波音位于华盛顿州和南卡罗来

#### 投资数据

In [17]:
invest_file = './data/invest-on-invent-KG.json'
with open(invest_file, 'r', encoding='utf-8') as file:
    invest_data = json.load(file)

##### 投资者数据

In [18]:
invest_data['@graph'][0]

{'@id': '0',
 '@type': 'investor',
 'name': '瑞华林投资',
 'relationship': {'investCompany': [{'@id': '5617',
    '@type': 'company',
    'round': '新三板定增',
    'date': '2016-03-04'}]}}

##### 公司信息

In [36]:
invest_data['@graph'][5323]

{'@id': '5323',
 '@type': 'company',
 'name': '福建博思软件股份有限公司',
 'alterNames': ['博思软件',
  '福建博思软件',
  '博思软件公司',
  '福建省博思软件',
  '福建博思软件公司',
  '福建省博思软件公司'],
 'establishDate': '2001-09-05',
 'address': '闽侯县上街镇高新大道5号',
 'relationship': {'applyPatent': [{'@id': '17314', '@type': 'patent'},
   {'@id': '17315', '@type': 'patent'},
   {'@id': '17316', '@type': 'patent'},
   {'@id': '17317', '@type': 'patent'},
   {'@id': '17318', '@type': 'patent'},
   {'@id': '17319', '@type': 'patent'},
   {'@id': '17320', '@type': 'patent'}]}}

##### 专利信息

In [20]:
invest_data['@graph'][141764]

{'@id': '141764',
 '@type': 'patent',
 'name': '一种基于DLL的软件架构',
 'patentType': '发明公布'}

### 数据处理

- 投资数据中：投资者和投资公司、公司和专利 id 的对应关系
- 投资数据的公司名称和事件数据名称的对应关系

In [15]:
obj_by_id = {} # key: id, value: object

for ele in invest_data['@graph']:
    obj_by_id[ele['@id']] = ele

#### 提取事件数据中的公司信息

{'content': '...',
 'label': [{'arguments': [{'时间': '29日'}, {'主体': '波音公司'}], 'event_type': '裁员'},
           {'arguments': [{'主体': '波音'}, {'数值': ['比去年同期下降近50%']}],
            'event_type': '主营业务收入减少'}],
 'title': '疫情冲击下波音将裁员10％，员工可选买断工龄或提前退休'}

如上所示，content.label[i].arguments[j].主体 的值为公司名称，需要提取出来

In [24]:
ignore_names = ['电器', '600万股', '公司', '135万元', '2018年', '新产业'] # 脏数据
event_type_company_names = {}
for data in event_type_data:
    for label in data['label']:
        if 'arguments' in label:
            for arg in label['arguments']:
                for key, value in arg.items():
                    if key == '主体':
                        if value in ignore_names:
                            continue
                        event_type_company_names[value] = 1

list(event_type_company_names.keys())[:10]

['小米集团',
 '小米',
 '格力电器',
 '波音公司',
 '波音',
 'GoPro',
 '得邦照明股份有限公司',
 '得邦照明',
 '横店集团得邦工程塑料有限公司',
 '汉邦高科']

#### 提取投资关系数据中的公司名称

{'@id': '6382',
 '@type': 'company',
 'name': '广州中设机器人智能装备股份有限公司',
 'alterNames': ['中设机器人智能装备',
  '中设机器人智能装备公司',
  '广州中设机器人智能装备',
  '广州市中设机器人智能装备',
  '广州中设机器人智能装备公司',
  '广州市中设机器人智能装备公司'],
 'establishDate': '2008-07-23',
 'address': '广州市黄埔区云埔工业区方达路6号101房',
 'relationship': {'applyPatent': []}}

 如上所示，当 @type 为 company 时，表示这个公司数据，需要提取 name 字段

In [25]:
invest_company_names = []
for ele in invest_data['@graph']:
    if ele['@type'] == "company":
        invest_company_names.append(ele['name'])

pprint(invest_company_names[:10])

['广州博鳌纵横网络科技有限公司',
 '福建博思软件股份有限公司',
 '深圳优地科技有限公司',
 '杭州群核信息技术有限公司',
 '昆山希盟自动化科技有限公司',
 '北京小爱智能科技有限公司',
 '奇安信科技集团股份有限公司',
 '北京大米科技有限公司',
 '北京诸葛找房信息技术有限公司',
 '苏州奥易克斯汽车电子有限公司']


#### 建立投资关系数据和事件数据中的公司关系

因为投资关系数据中的公司名称是完整的，所以以投资关系数据中的公司名称为基准，进行字符串包含判断，如果包含，则建立关系。

In [29]:
event_type_company_dict = {} # key: 事件关系中的公司名称；value: 投资数据中的公司名称列表
for company in event_type_company_names.keys():
    for iv_name in invest_company_names:
        if company in iv_name:
            event_type_company_dict.setdefault(company, []).append(iv_name) # 建立一对多关系

pprint(event_type_company_dict['小米'])
pprint(event_type_company_dict['瑞幸咖啡'])

['小米科技有限责任公司']
['瑞幸咖啡(北京)有限公司']


### 构建图数据库

#### 构建节点和边

- 公司和事件
- 公司和投资者
- 公司和专利

##### 公司和事件

```json
{
    "content": "...",
    "label": [
        {
            "arguments": [
                {
                    "时间": "29日"
                },
                {
                    "主体": "波音公司"
                }
            ],
            "event_type": "裁员"
        },
        {
            "arguments": [
                {
                    "主体": "波音"
                },
                {
                    "数值": [
                        "比去年同期下降近50%"
                    ]
                }
            ],
            "event_type": "主营业务收入减少"
        }
    ],
    "title": "疫情冲击下波音将裁员10％，员工可选买断工龄或提前退休"
}
```

In [32]:
for data in tqdm(event_type_data):
    title = data['title']
    content = data['content']

    for label in data['label']:
        event_type = label['event_type']
        event_type_node = Node("EventType", name=event_type)  # 建立事件类型节点
        graph.merge(event_type_node, "EventType", "name") # 添加到图数据库中，name 是主键属性的名称

        if 'arguments' in label:
            company_name = ''
            tmp_args = {}
            for arg in label['arguments']:
                for key, value in arg.items():
                    if key == '主体':
                        company_name = value
                    else:
                        tmp_args[key] = value
                if company_name in event_type_company_dict:
                    for cname in event_type_company_dict[company_name]:
                        company_node = Node("Company", name=cname) # 建立公司节点
                        graph.merge(company_node, "Company", "name")

                        relationship = Relationship(company_node, "HAPPEN", event_type_node) # 建立公司与事件的关系
                        for key ,value in tmp_args.items():
                            relationship[key] = value
                        relationship['title'] = title
                        relationship['content'] = content
                        graph.create(relationship) # 存储到图数据库中


100%|██████████| 7058/7058 [00:41<00:00, 169.19it/s]


##### 公司与专利

```json
{
    "@id": "5323",
    "@type": "company",
    "name": "福建博思软件股份有限公司",
    "alterNames": ["博思软件"],
    "establishDate": "2001-09-05",
    "address": "闽侯县上街镇高新大道5号",
    "relationship": {
        "applyPatent": [
            {
                "@id": "17314",
                "@type": "patent"
            }
        ]
    }
}
```

In [40]:
for ele in tqdm(invest_data['@graph']):
    if ele['@type'] == "company":
        name = ele['name']
        alter_names = ele['alterNames']
        establish_date = ele['establishDate']
        address = ele['address']
        apply_patent = ele['relationship']['applyPatent']

        company_node = Node("Company", name=name, establishDate=establish_date, address=address) # 建立公司节点
        graph.merge(company_node, "Company", "name")

        if len(apply_patent):
            for patent in apply_patent:
                patent_name = obj_by_id[patent['@id']]['name']
                patent_node = Node("Patent", name=patent_name) # 建立专利节点
                graph.merge(patent_node, "Patent", "name")

                relationship = Relationship(company_node, "HAVE", patent_node) # 公司与专利的关系
                graph.create(relationship)

  0%|          | 0/394204 [00:00<?, ?it/s]


##### 公司与投资者


```json
{
    "@id": "0",
    "@type": "investor",
    "name": "瑞华林投资",
    "relationship": {
        "investCompany": [
            {
                "@id": "5617",
                "@type": "company",
                "round": "新三板定增",
                "date": "2016-03-04"
            }
        ]
    }
}
```

In [ ]:
for ele in tqdm(invest_data['@graph']):
    if ele['@type'] == "investor":
        investor_node = Node("Investor", name=ele['name']) # 建立投资者节点
        graph.merge(investor_node, "Investor", "name")

        invest_company = ele['relationship']['investCompany']
        for invest in invest_company:
            acompany_ele = obj_by_id[invest['@id']]
            name = acompany_ele['name']
            alter_names = acompany_ele['alterNames']
            establish_date = acompany_ele['establishDate']
            address = acompany_ele['address']
            
            company_node = Node("Company", name=name, establishDate=establish_date, address=address) # 建立公司节点
            graph.merge(company_node, "Company", "name")

            relationship = Relationship(investor_node, "INVEST", company_node) # 建立投资者和公司的关系
            relationship['date'] = invest['date']
            relationship['round'] = invest['round']
            graph.create(relationship)